# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import torch
from collections import deque, namedtuple
from tensorboardX import SummaryWriter
from maddpg_agent import MADDPG

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
# for i in range(1, 6):                                      # play game for 5 episodes
#     env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
#     states = env_info.vector_observations                  # get the current state (for each agent)
#     scores = np.zeros(num_agents)                          # initialize the score (for each agent)
#     while True:
#         actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        
#         actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#         print('actions', actions)
#         env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#         next_states = env_info.vector_observations         # get next state (for each agent)
#         print('next_states',next_states.shape)
        
#         rewards = env_info.rewards                         # get reward (for each agent)
#         dones = env_info.local_done                        # see if episode finished
#         print('rewards',rewards.shape)
#         print('dones',dones.shape)
#         scores += env_info.rewards                         # update the score (for each agent)
#         states = next_states                               # roll over states to next time step
#         if np.any(dones):                                  # exit loop if episode finished
#             break
#         break
#     print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

When finished, you can close the environment.

In [6]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [7]:
def run(episodes,experiment_name, epsilon_start=1.0, epsilon_min=0.01, epsilon_decay=1e-5, max_score = 30.):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    epsilon = epsilon_start
    tb_writer = SummaryWriter('{}/{}'.format('logs',experiment_name))

    agent = MADDPG(num_agents=num_agents, state_size=state_size, action_size=action_size, capacity=int(1e5),
                   batch_size=256,learning_rate=0.001, update_rate=10,gamma=0.995, tau=0.001, device=device, seed=65, epsilon= epsilon_start)
    score_window = deque(maxlen=100)
    all_scores = []
    
    
    for episode in range(episodes):
        agent.reset()
        scores = np.zeros(num_agents) 
        dones = np.zeros((num_agents), dtype=bool)
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations 
        
        while not np.any(dones):
            actions = agent.act(states)
#             all actions between -1 and 1
            actions = np.clip(actions, -1, 1).cpu().numpy() 
    
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = np.array(env_info.rewards).reshape(2,1)                         # get reward (for each agent)
            dones = np.array(env_info.local_done).reshape(2,1)                        # see if episode finished
#             print(dones) 
#[False, False]
#             print(dones.shape, states.shape)
            scores += env_info.rewards                         # update the score (for each agent)
#             print(states.shape, actions.shape, rewards.shape, next_states.shape, dones.shape)
            agent.step(states, actions, rewards, next_states, dones)
            states = next_states
            
        
        episode_score = np.mean(scores)
        score_window.append(episode_score)
        all_scores.append(episode_score)
        
        #decay after each episode
        epsilon = max(epsilon_min, epsilon*epsilon_decay)
            
        print('\rEpisode: {}\tAverage Score: {}'.format(episode, np.mean(score_window)), end="")
        if episode % 100 == 0:
            print('\rEpisode: {}\tAverage Score: {}'.format(episode, np.mean(score_window)))
            tb_writer.add_scalar('Episode_Accum_score', np.mean(score_window), episode)

#         if np.mean(score_window) >= max_score:
#             torch.save(agent.actor_local_network.state_dict(), 'actor_checkpoint.pth')
#             torch.save(agent.critic_local_network.state_dict(), 'critic_checkpoint.pth')
#             break
            
run(1000, experiment_name="test" )

Episode: 0	Average Score: -0.004999999888241291
Episode: 13	Average Score: -0.0014285712635942868torch.Size([256, 48]) torch.Size([512, 2]) torch.Size([512, 1]) torch.Size([512, 24]) torch.Size([512, 1])
dones torch.Size([512, 1])
q_target_next torch.Size([512, 1])


RuntimeError: size mismatch, m1: [256 x 48], m2: [24 x 120] at /Users/soumith/code/builder/wheel/pytorch-src/aten/src/TH/generic/THTensorMath.cpp:2070